## **Serialbox Tutorial : Incorporating Serialbox Data from Fortran into Python**

In the [previous notebook](./01.ipynb), we covered how to extract data from a Fortran code using Serialbox.  In this notebook, we'll cover how to read and incorporate those files within a Python code.

### **Notebook Requirements**

- Python v3.11.x to v3.12.x
- [NOAA/NASA Domain Specific Language Middleware](https://github.com/NOAA-GFDL/NDSL)
- `ipykernel==6.1.0`
- [`ipython_genutils`](https://pypi.org/project/ipython_genutils/)

This notebook assumes that the code from the [previous notebook](./01.ipynb) was run, and the serialized data from Fortran was written out.

### **Importing Fortran Serialbox Data into Python** ###



In [1]:
import sys
# Appends the Serialbox path to PYTHONPATH.  If needed, change to appropriate path containing serialbox installation
sys.path.append('/home/ckung/Documents/Code/SMT-Nebulae/sw_stack_path/install/serialbox/python')
import serialbox as ser
import numpy as np

# If needed, change the path in second parameter of ser.Serializer to appropriate path that contains Fortran data via Serialbox from 01.ipynb
serializer_in = ser.Serializer(ser.OpenModeKind.Read,"./Fortran/sb/","FILLQ2ZERO_In")

savepoints = serializer_in.savepoint_list()

Qin_out = serializer_in.read("q", savepoints[0])
mass    = serializer_in.read("m", savepoints[0])
fq_out  = serializer_in.read("fq", savepoints[0])

print('Sum of Qin_out = ', sum(sum(sum(Qin_out))))
print('Sum of mass = ', sum(sum(sum(mass))))
print('Sum of fq_out = ', sum(sum(fq_out)))

Sum of Qin_out =  59.677578926086426
Sum of mass =  60.20569133758545
Sum of fq_out =  0.0


In [2]:
def fillq2zero1(Q, MASS, FILLQ):
    IM = Q.shape[0]
    JM = Q.shape[1]
    LM = Q.shape[2]

    TPW = np.sum(Q*MASS,2)
    for J in range(JM):
        for I in range(IM):
            NEGTPW = 0.
            for L in range(LM):
                if(Q[I,J,L] < 0.0):
                    NEGTPW = NEGTPW + (Q[I,J,L]*MASS[I,J,L])
                    Q[I,J,L] = 0.0
            for L in range(LM):
                if(Q[I,J,L] >= 0.0):
                    Q[I,J,L] = Q[I,J,L]*(1.0 + NEGTPW/(TPW[I,J]-NEGTPW))
            FILLQ[I,J] = -NEGTPW
            
fillq2zero1(Qin_out,mass,fq_out)

print('Sum of Qin_out = ', sum(sum(sum(Qin_out))))
print('Sum of fq_out = ', sum(sum(fq_out)))

Sum of Qin_out =  59.565123558044434
Sum of fq_out =  0.24599997873883694


In [4]:
# If needed, change the path in second parameter of ser.Serializer to appropriate path that contains Fortran data via Serialbox from 01.ipynb
serializer_out = ser.Serializer(ser.OpenModeKind.Read,"./Fortran/sb/","FILLQ2ZERO_Out")

savepoints_ref = serializer_out.savepoint_list()

Qin_out_ref = serializer_out.read("q_out", savepoints_ref[0])
mass_ref    = serializer_out.read("m_out", savepoints_ref[0])
fq_out_ref  = serializer_out.read("fq_out", savepoints_ref[0])

print(np.allclose(Qin_out,Qin_out_ref))
print(np.allclose(fq_out,fq_out_ref))

True
True
